# API demonstration for paper of v1.0

_the LSST-DESC CLMM team_


This notebook demonstrates how to use `clmm` to estimate a WL halo mass from observations of a galaxy cluster when source galaxies follow a given distribution (the Chang. (2013) implemented in `clmm`). It uses several functionalities of the support `mock_data` module to produce mock datasets.

- Setting things up, with the proper imports.
- Computing the binned reduced tangential shear profile, for the 2 datasets, using logarithmic binning.
- Setting up the "single source plane" model (model1) and a model accounting for the redshift distribution (model2). As already seen in Example2, model1 will yield a bias mass reconstruction. Accounting for the redshift distribution in the model (model2) solves that issue. 
- Perform a simple fit using `scipy.optimize.curve_fit` and visualize the results.

## Setup

First, we import some standard packages.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
plt.rcParams['font.family'] = ['gothambook','gotham','gotham-book','serif']

Next, we import `clmm`'s core modules.

In [ ]:
import clmm
import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.modeling as modeling
from clmm import Cosmology
clmm.__version__

## 1. Measuring shear profiles 

### 1.1. Making mock data for cluster members

In [ ]:
# Import mock module
import sys
sys.path.append('./support/')
import mock_data as mock
from sampler import fitters
np.random.seed(11) # For reproducibility

# Set cosmology of mock data
cosmo = Cosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)

# Cluster info
cluster_m = 1.e15 # Cluster mass - (M200_m) [Msun]
concentration = 4  # Cluster concentration
cluster_z = 0.4 # Cluster redshift
cluster_ra = 0. # Cluster Ra in deg
cluster_dec = 0. # Cluster Dec in deg

# Make mock galaxies
mock_galaxies = mock.generate_galaxy_catalog(
    cluster_m=cluster_m, cluster_z=cluster_z, cluster_c=concentration, # Cluster data
    cosmo=cosmo, # Cosmology object
    zsrc='chang13', # Galaxy redshift distribution, 
    zsrc_min=0.5, # Minimum redshift of the galaxies
    shapenoise=0.05, # Gaussian shape noise to the galaxy shapes
    photoz_sigma_unscaled=0.05, # Photo-z errors to source redshifts
    ngals=10000 # Number of galaxies to be generated
)['ra', 'dec', 'e1', 'e2', 'z', 'ztrue', 'pzbins', 'pzpdf', 'id']#['ra, dec, e1, e2, z, ztrue, pzbins, pzpdf, id']
print(f'This results in a table with the columns: {", ".join(mock_galaxies.colnames)}')

# Put galaxy values on arrays
gal_ra = mock_galaxies['ra'] # Galaxies Ra in Deg
gal_dec = mock_galaxies['dec'] # Galaxies Dec in Deg
gal_e1 = mock_galaxies['e1'] # Galaxies elipticipy 1
gal_e2 = mock_galaxies['e2'] # Galaxies elipticipy 2
gal_z = mock_galaxies['z'] # Galaxies observed redshift
gal_ztrue = mock_galaxies['ztrue'] # Galaxies true redshift
gal_pzbins = mock_galaxies['pzbins'] # Galaxies P(z) bins  
gal_pzpdf = mock_galaxies['pzpdf'] # Galaxies P(z)
gal_id = mock_galaxies['id'] # Galaxies ID

### 1.2. Measure profile with explicit functions

In [ ]:
# Convert elipticities into shears
gal_ang_dist, gal_gt, gal_gx = da.compute_tangential_and_cross_components(cluster_ra, cluster_dec,
                                                                          gal_ra, gal_dec,
                                                                          gal_e1, gal_e2,
                                                                          geometry="flat")

# Measure profile
profile = da.make_radial_profile([gal_gt, gal_gx, gal_z],
                                 gal_ang_dist, "radians", "Mpc",
                                 bins=da.make_bins(0.01, 3.7, 50),
                                 cosmo=cosmo,
                                 z_lens=cluster_z,
                                 include_empty_bins=False)
print(f'Profile table has columns: {", ".join(profile.colnames)},')
print('where p_(0, 1, 2) = (gt, gx, z)')

### 1.3. Measure profile with GalaxyCluster object

In [ ]:
# Create a GCData with the galaxies
galaxies = clmm.GCData()
galaxies['ra'] = gal_ra
galaxies['dec'] = gal_dec
galaxies['e1'] = gal_e1
galaxies['e2'] = gal_e2
galaxies['z'] = gal_z
galaxies['ztrue'] = gal_ztrue
galaxies['pzbins'] = gal_pzbins
galaxies['pzpdf'] = gal_pzpdf
galaxies['id'] = gal_id

# Create a GalaxyCluster
cluster = clmm.GalaxyCluster("Name of cluster", cluster_ra, cluster_dec,
                                   cluster_z, noisy_data)

# Convert elipticities into shears for the members
cluster.compute_tangential_and_cross_components(geometry="flat")
print(cluster.galcat.colnames)

# Measure profile and add profile table to the cluster
cluster.make_radial_profile(bins=da.make_bins(0.01, 3.7, 50),
                            bin_units="Mpc",
                            cosmo=cosmo,
                            include_empty_bins=False)
print(cluster.profile.colnames)

## 2. Theoretical prediction

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(mock_galaxies['z'], density = True, bins = 50)
plt.axvline(x = cluster_z, color='orange', label = 'cluster redshift')
plt.xlabel(r'$z_{src}$', fontsize = 20)
plt.ylabel(r'$N(z$)', fontsize = 20)
plt.legend()
plt.xlim(0,5)

## Passing data to CLMM objects

In [ ]:
# create a galaxy catalog with GCData class
galaxies = GCData()

The galaxy catalogs are converted to a `clmm.GalaxyCluster` object and may be saved for later use.

In [ ]:
cluster_id = "CL_ideal"
gc_object = clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec,
                                  cluster_z, ideal_data)
gc_object.save('ideal_GC.pkl')

cluster_id = "CL_noisy"
gc_object = clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec,
                                   cluster_z, noisy_data)
gc_object.save('noisy_GC.pkl')

Any saved clmm.GalaxyCluster object may be read in for analysis.

In [ ]:
cl_ideal = clmm.GalaxyCluster.load('ideal_GC.pkl') # background galaxies distributed according to Chang et al. (2013)
cl_noisy = clmm.GalaxyCluster.load('noisy_GC.pkl') # Chang et al. (2013) + shapenoise + photozerror

#### Redshift of galaxies generated by mock data are distributed following the Chang. (2013) redshift distribution.

## Deriving observables

## Create the reduced tangential shear models

We consider two options:
- First, the naive and *wrong* approach: the reduced tangential shear in a given radial bin $j$ is given by $g_t(\theta_j, \langle z_s \rangle)$, where $\langle z_s \rangle$ is the average redshift in the bin. In that case, the corresponding model is simply given by the fucntion below:

In [ ]:
def model_reduced_tangential_shear_singlez(r, logm, z_src):
    m = 10.**logm
    gt_model = clmm.predict_reduced_tangential_shear(r,
                                                     m, concentration,
                                                     cluster_z, z_src, cosmo,
                                                     delta_mdef=200,
                                                     halo_profile_model='nfw')    
    return gt_model

- Second, the reduced tangential shear in a given radial bin accounts properly for the redshift distribution in the bin as $\langle g_t(\theta_j, z_s)\rangle \neq g_t(\theta_j, \langle z_s \rangle$). Formally, the reduced tangential shear that corresponds to a continuous distribution of source galaxy redshift $N(z)$ can be expressed as:

$$
g_t(\theta) = \langle g_t(\theta, z_s)\rangle_{z_{cluster}} = \int_{z_{cluster}}^{+\infty}dz_sN(z_s)g_t(\theta, z_s)
$$

If the inidividual redshifts of the background galaxies are known, we can directly build a model based on data, such that in the bin $j$: 

$$
g_t(\theta_j) = \frac{1}{N(\theta_j)}\sum\limits_{i = 1}^{N(\theta)}g_t(\theta_j, z_i)
$$

where $N(\theta_j)$ is the number of galaxies in bin $j$. The corresponding model is given below.

In [ ]:
def model_reduced_tangential_shear_zdistrib(radius, logm, data, catalog, profile): 
    m = 10**logm
    gt_model = []
    for i in range(len(radius)):
        
        r = profile['radius'][i]
        galist = profile['gal_id'][i]
        z_list = catalog.galcat['z'][galist]
        shear = clmm.predict_reduced_tangential_shear(r, m, concentration, 
                                                      cluster_z, z_list, cosmo, delta_mdef=200, 
                                                      halo_profile_model='nfw')
        gt_model.append(np.mean(shear))
        
    return gt_model

#### Before fitting, let's first vizualise these models using the known true mass

In [ ]:
logm = np.log10(cluster_m)
r = cl_ideal.profile['radius']
gt_model_ideal_singlez = model_reduced_tangential_shear_singlez(r, logm, cl_ideal.profile['z'])
gt_model_ideal_zdistrib = model_reduced_tangential_shear_zdistrib(r,logm, ideal_data, cl_ideal, cl_ideal.profile)
gt_model_noisy_singlez = model_reduced_tangential_shear_singlez(r,logm, cl_noisy.profile['z'])
gt_model_noisy_zdistrib = model_reduced_tangential_shear_zdistrib(r,logm, noisy_data, cl_noisy, cl_noisy.profile)

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)

plt.title('ideal data', fontsize=20)
plt.errorbar(r,cl_ideal.profile['gt'],cl_ideal.profile['gt_err'],c='k',linestyle='', 
             marker='o', label=r'ideal data, $M_{input}$ = %.2e Msun' % cluster_m)
plt.loglog(r,gt_model_ideal_zdistrib,'b',  label=r'model w/ zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
plt.loglog(r,gt_model_ideal_singlez,'-y',  label=r'model w/o zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
plt.xlabel('r [Mpc]', fontsize = 20)
plt.ylabel(r'$g_t$', fontsize = 20)
plt.xlim(min(cl_ideal.profile['radius']), max(cl_ideal.profile['radius']))
plt.legend(fontsize = 15)

plt.subplot(1,2,2)

plt.title('noisy data', fontsize=20)
plt.errorbar(r,cl_noisy.profile['gt'],cl_noisy.profile['gt_err'],c='k',linestyle='', 
             marker='o',label=r'noisy data, $M_{input}$ = %.2e Msun' % cluster_m)
plt.loglog(r,gt_model_noisy_zdistrib,'-b', label=r'model w/ zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
plt.loglog(r,gt_model_noisy_singlez,'-y', label=r'model w/o zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
plt.xlabel('r [Mpc]', fontsize = 20)
plt.ylabel(r'$g_t$', fontsize = 20)
plt.xlim(min(cl_noisy.profile['radius']), max(cl_noisy.profile['radius']))
plt.legend(fontsize = 15)

The naive model that uses the average redshift in the bin clearly does not give the right description of the ideal data (left panel), and will yield biased mass results if used for fitting (see below). For ideal data, the model that accounts for the redshift distribution is, by construction, an excellent description of the data (solid blue line). The same is true for noisy data (right panel), although the noise make the naive model appear "less biased".

## Mass fitting

We estimate the best-fit mass using `scipy.optimize.curve_fit`.  We compare estimated mass for noisy and ideal data, using both models described above (naive with average redshift or the model taking into account the redshift distribution). The choice of fitting $\log_{10} M$ instead of $M$ lowers the range of pre-defined fitting bounds from several order of magnitude for the mass to unity. From the associated error $\Delta (\log_{10}M)$ we calculate the error to mass as $\Delta M = M_{fit}\log(10)\Delta (\log_{10}M)$.

In [ ]:
popt,pcov = fitters['curve_fit'](lambda r, logm:model_reduced_tangential_shear_zdistrib(r, logm, ideal_data, cl_ideal, cl_ideal.profile), 
                        cl_ideal.profile['radius'], 
                        cl_ideal.profile['gt'], 
                        cl_ideal.profile['gt_err'], bounds=[10.,16.])

m_est_ideal_zdistrib = 10.**popt[0]
m_est_err_ideal_zdistrib =  m_est_ideal_zdistrib * np.sqrt(pcov[0][0]) * np.log(10) 

popt,pcov = fitters['curve_fit'](lambda r, logm:model_reduced_tangential_shear_singlez(r, logm, cl_ideal.profile['z']), 
                        cl_ideal.profile['radius'], 
                        cl_ideal.profile['gt'], 
                        cl_ideal.profile['gt_err'], bounds=[10.,17.])

m_est_ideal_singlez = 10.**popt[0]
m_est_err_ideal_singlez =  m_est_ideal_singlez * np.sqrt(pcov[0][0]) * np.log(10) 


popt,pcov = fitters['curve_fit'](lambda r, logm:model_reduced_tangential_shear_zdistrib(r, logm, noisy_data, cl_noisy, cl_noisy.profile), 
                        cl_noisy.profile['radius'], 
                        cl_noisy.profile['gt'], 
                        cl_noisy.profile['gt_err'], bounds=[10.,16.])

m_est_noisy_zdistrib = 10.**popt[0]
m_est_err_noisy_zdistrib =  m_est_noisy_zdistrib * np.sqrt(pcov[0][0]) * np.log(10) 

popt,pcov = fitters['curve_fit'](lambda r, logm:model_reduced_tangential_shear_singlez(r, logm, cl_noisy.profile['z']), 
                        cl_noisy.profile['radius'], 
                        cl_noisy.profile['gt'], 
                        cl_noisy.profile['gt_err'], bounds=[10.,16.])

m_est_noisy_singlez = 10.**popt[0]
m_est_err_noisy_singlez =  m_est_noisy_singlez * np.sqrt(pcov[0][0]) * np.log(10) 


In [ ]:
print(f'The input mass = {cluster_m:.2e} Msun\n')

print("Without accounting for the redshift distribution in the model\n")
print(f'Best fit mass for ideal data = {m_est_ideal_singlez:.2e} +/- {m_est_err_ideal_singlez:.2e} Msun')
print(f'Best fit mass for noisy data = {m_est_noisy_singlez:.2e} +/- {m_est_err_noisy_singlez:.2e} Msun\n')

print("Accounting for the redshift distribution in the model\n")
print(f'Best fit mass for ideal data = {m_est_ideal_zdistrib:.2e} +/- {m_est_err_ideal_zdistrib:.2e} Msun')
print(f'Best fit mass for noisy data = {m_est_noisy_zdistrib:.2e} +/- {m_est_err_noisy_zdistrib:.2e} Msun')

As expected, the reconstructed mass is biased when the redshift distribution is not accounted for in the model

# Visualization of the results

For visualization purpose, we calculate the reduced tangential shear predicted by the model with estimated masses for noisy and ideal data.

In [ ]:
gt_est_ideal_zdistrib = model_reduced_tangential_shear_zdistrib(r,np.log(m_est_ideal_zdistrib)/np.log(10), ideal_data, cl_ideal, cl_ideal.profile)
gt_est_noisy_zdistrib = model_reduced_tangential_shear_zdistrib(r,np.log(m_est_noisy_zdistrib)/np.log(10), noisy_data, cl_noisy, cl_noisy.profile)
gt_est_ideal_singlez = model_reduced_tangential_shear_singlez(r,np.log(m_est_ideal_singlez)/np.log(10),  cl_ideal.profile['z'])
gt_est_noisy_singlez = model_reduced_tangential_shear_singlez(r,np.log(m_est_noisy_singlez)/np.log(10),  cl_noisy.profile['z'])

We compare to tangential shear obtained with theoretical mass. We plot the reduced tangential shear models first when redshift distribution is accounted for in the model then for the naive approach, with respective best-fit masses.

In [ ]:
plt.figure(figsize=( 20 , 6 ))
plt.subplot( 1 , 2 , 1 )
plt.title(r'tangential shear $g_t$ (ideal data)', fontsize=20)
plt.errorbar(r,cl_ideal.profile['gt'],cl_ideal.profile['gt_err'],c='k',linestyle='', 
             marker='o', label=r'ideal data, $M_{input}$ = %.1e Msun' % cluster_m)
plt.loglog(r,gt_est_ideal_zdistrib,'-b', 
           label=fr'model w/ zdistrib, M_fit = {m_est_ideal_zdistrib:.2e} $\pm$ {m_est_err_ideal_zdistrib:.2e} Msun')
plt.loglog(r,gt_est_ideal_singlez,'-y',\
           label=fr'model w/o zdistrib, M_fit = {m_est_ideal_singlez:.2e} $\pm$ {m_est_err_ideal_singlez:.2e} Msun')

plt.xlabel('r [Mpc]', fontsize = 20)
plt.ylabel(r'$g_t$', fontsize = 20)
plt.xlim(min(cl_ideal.profile['radius']), max(cl_ideal.profile['radius']))
plt.legend(fontsize = 15)


plt.subplot( 1 , 2 , 2 )
plt.title(r'tangential shear $g_t$ (noisy data)', fontsize=20)
plt.errorbar(r,cl_noisy.profile['gt'],cl_noisy.profile['gt_err'],c='k',linestyle='', marker='o', label=r'noisy data, $M_{input}$ = %.1e Msun' % cluster_m)
#plt.loglog(r,gt_model_noisy,'-r',  label='model, $M_{input}$ = %.3e Msun' % cluster_m)
plt.loglog(r,gt_est_noisy_zdistrib,'-b', 
           label=fr'model w/ zdistrib, M_fit = {m_est_noisy_zdistrib:.2e} $\pm$ {m_est_err_noisy_zdistrib:.2e} Msun')
plt.loglog(r,gt_est_noisy_singlez,'-y', 
           label=fr'model w/o zdistrib, M_fit = {m_est_noisy_singlez:.2e} $\pm$ {m_est_err_noisy_singlez:.2e} Msun')

plt.xlabel('r [Mpc]', fontsize = 20)
plt.ylabel(r'$g_t$', fontsize = 20)
plt.xlim(min(cl_noisy.profile['radius']), max(cl_noisy.profile['radius']))
plt.legend(fontsize = 15)

We note that the reconstruction of mass is biaised when redshift distribution is not accounted for the model, and is smaller compared to input mass. It is associated to the increase of the reduced tangential shear with the source redshift $z_s$ for a given radius $r$. 